In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os # For listing files
import warnings

# Configure pandas to display more rows/columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

data_path = r"C:\Users\HP\Downloads\archive"
#listing CSV files in the dataset
print("CSV files found in the raw data folder:")
for file_name in os.listdir(data_path):
    if file_name.endswith('.csv'):
        print(f"- {file_name}")

## Loading each CSV file into a separate pandas DataFrame
customers_df = pd.read_csv(os.path.join(data_path, 'olist_customers_dataset.csv'))
orders_df = pd.read_csv(os.path.join(data_path, 'olist_orders_dataset.csv'))
order_items_df = pd.read_csv(os.path.join(data_path, 'olist_order_items_dataset.csv'))
order_payments_df = pd.read_csv(os.path.join(data_path, 'olist_order_payments_dataset.csv'))
products_df = pd.read_csv(os.path.join(data_path, 'olist_products_dataset.csv'))
order_reviews_df = pd.read_csv(os.path.join(data_path, 'olist_order_reviews_dataset.csv'))
product_category_translation_df = pd.read_csv(os.path.join(data_path, 'product_category_name_translation.csv'))

CSV files found in the raw data folder:
- olist_customers_dataset.csv
- olist_geolocation_dataset.csv
- olist_orders_dataset.csv
- olist_order_items_dataset.csv
- olist_order_payments_dataset.csv
- olist_order_reviews_dataset.csv
- olist_products_dataset.csv
- olist_sellers_dataset.csv
- product_category_name_translation.csv


In [4]:
# Cleaning orders df

# Convert all order timestamp columns to datetime objects
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_columns:
    # 'errors='coerce' will turn any dates that can't be parsed into NaT (Not a Time), which is like NaN for dates.
    orders_df[col] = pd.to_datetime(orders_df[col], errors='coerce')

# Checking how many missing values are in the delivery date column after conversion
print("\nMissing 'order_delivered_customer_date' after conversion:")
print(orders_df['order_delivered_customer_date'].isnull().sum())

# For our analysis, we are most interested in *completed* orders.
# 'delivered' orders are those that actually reached the customer.
print(orders_df['order_status'].value_counts())

# Filtering the orders_df to only include 'delivered' orders.
delivered_orders_df = orders_df[orders_df['order_status'] == 'delivered'].copy()
print(delivered_orders_df.shape)

# Also check for any missing customer_id in these delivered orders, as customer_id is crucial.
print(delivered_orders_df['customer_id'].isnull().sum())

# If there are any missing 'customer_id' in delivered orders, it's best to drop them for CLTV.
# 'inplace=True' means the change happens directly to delivered_orders_df
if delivered_orders_df['customer_id'].isnull().sum() > 0:
    delivered_orders_df.dropna(subset=['customer_id'], inplace=True)
    print(delivered_orders_df.shape)



Missing 'order_delivered_customer_date' after conversion:
2965
order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64
(96478, 8)
0
